# 第三章第一节：生成器：参数


## 本节的目的

要使Chisel模块成为代码生成器，必须有一些东西可以告诉生成器应该如何处理它的工作。
在本节中，我们将讨论模块的参数化，参数化的各种方法以及Scala语言特性。
能够靠参数传递实现的丰富程度与其生成的电路的丰富程度直接相关。
参数应提供有用的默认值，易于设置，并防止非法或无意义的值。
对于更复杂的系统，如果它们可以在局部被覆盖（override），而不会无意中影响其他模块中的使用，则非常有用。

## 设置

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# 参数传递

Chisel为编写硬件生成器提供了强大的语法。
生成器可以接收一些电路参数并生成电路描述的程序。
在本节中，我们将首先讨论Chisel生成器如何获取其参数。

<span style =“color：blue”> **示例：参数化的Scala对象** </ span> <br>
每个Chisel`模块`都是Scala类，就像其他任何类一样。
回想一下，Scala类可以像这样参数化：

In [3]:
class ParameterizedScalaObject(param1: Int, param2: String) {
  println(s"I have parameters: param1 = $param1 and param2 = $param2")
}
val obj1 = new ParameterizedScalaObject(4,     "Hello")
val obj2 = new ParameterizedScalaObject(4 + 2, "World")

I have parameters: param1 = 4 and param2 = Hello
I have parameters: param1 = 6 and param2 = World


defined class ParameterizedScalaObject
obj1: ParameterizedScalaObject = ammonite.$sess.cmd2$Helper$ParameterizedScalaObject@74e68d48
obj2: ParameterizedScalaObject = ammonite.$sess.cmd2$Helper$ParameterizedScalaObject@5231ed71

<span style ="ccolor：blue"> **示例：参数化Chisel对象** </span> <br>
Chisel模块可以以相同的方式参数化。
以下模块的所有输入和输出都包含了作为其宽度的参数。
运行下面的代码将打印生成的Verilog。
尝试一下别的参数并看看生成的结果有什么不同吧：

In [4]:
class ParameterizedWidthAdder(in0Width: Int, in1Width: Int, sumWidth: Int) extends Module {
  require(in0Width >= 0)
  require(in1Width >= 0)
  require(sumWidth >= 0)
  val io = IO(new Bundle {
    val in0 = Input(UInt(in0Width.W))
    val in1 = Input(UInt(in1Width.W))
    val sum = Output(UInt(sumWidth.W))
  })
  // a +& b 包含进位, 而 a + b 不包含
  io.sum := io.in0 +& io.in1
}

println(getVerilog(new ParameterizedWidthAdder(1, 4, 6))) // 以宽度 1，4，6 生成电路

[info] [0.002] Elaborating design...
[info] [0.839] Done elaborating.
Total FIRRTL Compile Time: 596.6 ms
module cmd3HelperParameterizedWidthAdder(
  input        clock,
  input        reset,
  input        io_in0,
  input  [3:0] io_in1,
  output [5:0] io_sum
);
  wire [3:0] _GEN_0; // @[cmd3.sc 11:20]
  wire [4:0] _T; // @[cmd3.sc 11:20]
  assign _GEN_0 = {{3'd0}, io_in0}; // @[cmd3.sc 11:20]
  assign _T = _GEN_0 + io_in1; // @[cmd3.sc 11:20]
  assign io_sum = {{1'd0}, _T}; // @[cmd3.sc 11:10]
endmodule



defined class ParameterizedWidthAdder


上面的代码块包含一些`require（...）`语句。
这些是在电路展开（elaboration）之前运行的断言，当您的生成器仅适用于某些参数化或者某些参数是互斥或相关时，这些断言非常有用。
上面的代码检查宽度是否为负数。

在电路仿真时（simulation）的断言叫做`assert（...）`。

## 参数化的排序模块

下面的代码块是一个参数化的排序模块，类似于2.3节中的`Sort4`。
与前面带参数化宽度IO的加法器示例不同，此示例具有固定IO。
该参数控制模块内部生成的硬件。
![Sort4](images/Sorter4.png)
<span style="color:blue"> **示例：参数化的四输入排序** </span><br>
与2.3节不同的是，这里的实现被参数化为降序或升序。

In [5]:
/** Sort4 接收 4 个输入，4 个输出为排序后的结果 */
class Sort4(ascending: Boolean) extends Module {
  val io = IO(new Bundle {
    val in0 = Input(UInt(16.W))
    val in1 = Input(UInt(16.W))
    val in2 = Input(UInt(16.W))
    val in3 = Input(UInt(16.W))
    val out0 = Output(UInt(16.W))
    val out1 = Output(UInt(16.W))
    val out2 = Output(UInt(16.W))
    val out3 = Output(UInt(16.W))
  })
    
  // 这个比较函数根据参数来决定是升序还是降序
  def comp(l: UInt, r: UInt): Bool = {
      if (ascending) {
        l < r
      } else {
        l > r
    }
  }

  val row10 = Wire(UInt(16.W))
  val row11 = Wire(UInt(16.W))
  val row12 = Wire(UInt(16.W))
  val row13 = Wire(UInt(16.W))

  when(comp(io.in0, io.in1)) {
    row10 := io.in0            // 保留前两个元素的顺序
    row11 := io.in1
  }.otherwise {
    row10 := io.in1            // 交换前两个元素的顺序
    row11 := io.in0
  }

  when(comp(io.in2, io.in3)) {
    row12 := io.in2            // 保留后两个元素的顺序
    row13 := io.in3
  }.otherwise {
    row12 := io.in3            // 交换后两个元素的顺序
    row13 := io.in2
  }

  val row21 = Wire(UInt(16.W))
  val row22 = Wire(UInt(16.W))

  when(comp(row11, row12)) {
    row21 := row11            // 保留中间两个元素的顺序
    row22 := row12
  }.otherwise {
    row21 := row12            // 交换中间两个元素的顺序
    row22 := row11
  }

  val row31 = Wire(UInt(16.W))
  val row32 = Wire(UInt(16.W))
  when(comp(row10, row13)) {
    row31 := row10            // 保留中间两个元素的顺序
    row32 := row13
  }.otherwise {
    row31 := row13            // 交换中间两个元素的顺序
    row32 := row10
  }

  when(comp(row10, row21)) {
    io.out0 := row31            // 保留前两个元素的顺序
    io.out1 := row21
  }.otherwise {
    io.out0 := row21            // 交换前两个元素的顺序
    io.out1 := row31
  }

  when(comp(row22, row13)) {
    io.out2 := row22            // 保留后两个元素的顺序
    io.out3 := row32
  }.otherwise {
    io.out2 := row32            // 交换后两个元素的顺序
    io.out3 := row22
  }
}

// 测试
class Sort4AscendingTester(c: Sort4) extends PeekPokeTester(c) {
  poke(c.io.in0, 3)
  poke(c.io.in1, 6)
  poke(c.io.in2, 9)
  poke(c.io.in3, 12)
  expect(c.io.out0, 3)
  expect(c.io.out1, 6)
  expect(c.io.out2, 9)
  expect(c.io.out3, 12)

  poke(c.io.in0, 13)
  poke(c.io.in1, 4)
  poke(c.io.in2, 6)
  poke(c.io.in3, 1)
  expect(c.io.out0, 1)
  expect(c.io.out1, 4)
  expect(c.io.out2, 6)
  expect(c.io.out3, 13)

  poke(c.io.in0, 13)
  poke(c.io.in1, 6)
  poke(c.io.in2, 4)
  poke(c.io.in3, 1)
  expect(c.io.out0, 1)
  expect(c.io.out1, 4)
  expect(c.io.out2, 6)
  expect(c.io.out3, 13)

}
class Sort4DescendingTester(c: Sort4) extends PeekPokeTester(c) {
  poke(c.io.in0, 3)
  poke(c.io.in1, 6)
  poke(c.io.in2, 9)
  poke(c.io.in3, 12)
  expect(c.io.out0, 12)
  expect(c.io.out1, 9)
  expect(c.io.out2, 6)
  expect(c.io.out3, 3)

  poke(c.io.in0, 13)
  poke(c.io.in1, 4)
  poke(c.io.in2, 6)
  poke(c.io.in3, 1)
  expect(c.io.out0, 13)
  expect(c.io.out1, 6)
  expect(c.io.out2, 4)
  expect(c.io.out3, 1)
    
  poke(c.io.in0, 1)
  poke(c.io.in1, 6)
  poke(c.io.in2, 4)
  poke(c.io.in3, 13)
  expect(c.io.out0, 13)
  expect(c.io.out1, 6)
  expect(c.io.out2, 4)
  expect(c.io.out3, 1)

}

// 测试
val worksAscending = iotesters.Driver(() => new Sort4(true)) { c => new Sort4AscendingTester(c) }
val worksDescending = iotesters.Driver(() => new Sort4(false)) { c => new Sort4DescendingTester(c) }
assert(worksAscending && worksDescending) // Scala 代码: 如果 works == false 的话会抛出异常
println("SUCCESS!!") // Scala 代码: 到这里的话表示测试通过

[info] [0.000] Elaborating design...
[info] [0.082] Done elaborating.
Total FIRRTL Compile Time: 121.1 ms
Total FIRRTL Compile Time: 64.7 ms
End of dependency graph
Circuit state created
[info] [0.003] SEED 1572042778423
test cmd4HelperSort4 Success: 12 tests passed in 5 cycles taking 0.044866 seconds
[info] [0.020] RAN 0 CYCLES PASSED
[info] [0.000] Elaborating design...
[info] [0.010] Done elaborating.
Total FIRRTL Compile Time: 66.2 ms
Total FIRRTL Compile Time: 63.0 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1572042779126
test cmd4HelperSort4 Success: 12 tests passed in 5 cycles taking 0.021682 seconds
[info] [0.016] RAN 0 CYCLES PASSED
SUCCESS!!


defined class Sort4
defined class Sort4AscendingTester
defined class Sort4DescendingTester
worksAscending: Boolean = true
worksDescending: Boolean = true

---
# 可选参数和默认参数


有时函数有时会返回一个值，有时则不会。 Scala在类型系统中提供了一种机制可以处理这样的情况，而不是在无法返回值时发生错误。

<span style="color:blue">**示例：错误的映射（map）索引调用** </span><br>
在以下示例中，我们有一个包含多个键/值对的映射。如果我们尝试访问缺少的键/值对，那么我们会得到运行时错误：

In [6]:
val map = Map("a" -> 1)
val a = map("a")
println(a)
val b = map("b")
println(b)

1


: 


<span style="color:blue">**示例：获取不确定是否存在的索引**</span><br>
但是，`Map`通过 **get**方法提供了另一种访问键值的方法。它会返回一个抽象类`Option`的实例。而`Option`有两个子类，`Some`和`None`。

In [7]:
val map = Map("a" -> 1)
val a = map.get("a")
println(a)
val b = map.get("b")
println(b)

Some(1)
None


map: Map[String, Int] = Map("a" -> 1)
a: Option[Int] = Some(1)
b: Option[Int] = None


正如您将在后面的章节中看到的那样，`Option`非常重要，因为它允许用户使用match语句来检查Scala类型和值。

<span style="color:blue">**示例：Get Or Else！** </span><br>
就像`Map`一样，`Option`也有一个`get`方法，如果其为`None`时调用就会出错。对于这些实例，我们可以使用 **`getOrElse`** 为其提供默认值。

In [8]:
val some = Some(1)
val none = None
println(some.get)          // 返回 1
// println(none.get)       // 错误!
println(some.getOrElse(2)) // 返回 1
println(none.getOrElse(2)) // 返回 2

1
1
2


some: Some[Int] = Some(1)
none: None.type = None

## 默认参数中使用Option


当对象或函数具有大量参数时，如果每次都需要指定全部参数可能会非常繁琐且容易出错。
在第一章中，您已经了解了命名参数和参数默认值。
有时，参数并没有良好的默认值。
在这样的情况下，`Option`可以使用默认值`None`。

<span style="color:blue">**示例：可选重置** </span><br>
以下的代码将输入延迟一个时钟周期。
如果`resetValue = None`（这是默认值），寄存器将没有复位值并被初始化为垃圾。
这避免了使用正常范围之外的值来表示“无”这样的情况，例如使用-1作为复位值来表示该寄存器未被重置。

In [9]:
class DelayBy1(resetValue: Option[UInt] = None) extends Module {
    val io = IO(new Bundle {
        val in  = Input( UInt(16.W))
        val out = Output(UInt(16.W))
    })
    val reg = if (resetValue.isDefined) { // resetValue = Some(number)
        RegInit(resetValue.get)
    } else { //resetValue = None
        Reg(UInt())
    }
    reg := io.in
    io.out := reg
}

println(getVerilog(new DelayBy1))
println(getVerilog(new DelayBy1(Some(3.U))))

[info] [0.000] Elaborating design...
[info] [0.053] Done elaborating.
Total FIRRTL Compile Time: 57.3 ms
module cmd8HelperDelayBy1(
  input         clock,
  input         reset,
  input  [15:0] io_in,
  output [15:0] io_out
);
  reg [15:0] reg_; // @[cmd8.sc 9:12]
  reg [31:0] _RAND_0;
  assign io_out = reg_; // @[cmd8.sc 12:12]
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DELAY begin end
      `else
        #0.002 begin end
      `endif
    `endif
  `ifdef RANDOMIZE_REG_INIT
  _RAND_0 = {1{`RANDOM}};
  reg_ = _RAND_0[15:0];
  `endif // RANDOMIZE_REG_INIT
  `endif // R

defined class DelayBy1

---
# Match/Case 语句

在Scala中，*match*的概念被用在整个Chisel的设计中，这是每个Chisel程序员必须理解的一部分。 Scala提供match运算符，它支持：
 - 对几个可能性进行简单测试，类似C语言的*switch*语句
 - 对特定值的组合的复杂测试
 - 当变量的类型未知或未指定时，根据变量的类型执行操作，例如：
   - 变量来自列表```val mixedList = List(1, "string", false)```
   - 或者已知变量是超类的实例，但不知道它是哪个特定的子类。
 - 提取用*正则表达式*指定的字符串的子字符串


<span style="color:blue"> **示例：值匹配** </span><br>
下面的例子，根据我们**匹配**的变量的**值**，我们执行不同的**case**语句：

In [10]:
// y 是在代码中其他地方定义的整数变量
val y = 7
/// ...
val x = y match {
  case 0 => "zero" // 常见的语法，如果一行代码能放下的话，优先使用这种语法
  case 1 =>        // 另一种常见的语法，如果一行放不下的话使用
      "one"        // 注意：代码块一直到下一个 case 语句为止
  case 2 => {      // 另一种语法， 但是大括号不是必需的
      "two"
  }
  case _ => "many" // _ 是可以匹配任何值的通配符|
}
println("y is " + x)

y is many


y: Int = 7
x: String = "many"


上面的代码中，match操作符为每个case检查可能的值，并返回一个字符串。有几点需要注意：
 - 在```=>```运算符后面的每个代码块一直持续到它遇到match结束的大括号或下一个case语句为止。
 - 按case语句的顺序进行匹配，一旦匹配了某个case语句，就结束了
 - 使用下划线作为通配符，以处理未匹配的其他值。

<span style="color:blue">**示例：多值匹配** </span><br>
此外，还可以同时匹配多个变量。这是一个使用匹配语句实现的真值表（truth table）的简单示例：

In [11]:
def animalType(biggerThanBreadBox: Boolean, meanAsCanBe: Boolean): String = {
  (biggerThanBreadBox, meanAsCanBe) match {
    case (true, true) => "wolverine"
    case (true, false) => "elephant"
    case (false, true) => "shrew"
    case (false, false) => "puppy"
  }
}
println(animalType(true, true))

wolverine


defined function animalType

<span style="color:blue">**示例：类型匹配** </span><br>
Scala是一种强类型语言，因此所有对象的类型在运行时都是已知的。我们可以使用**match语句**来使用此类型信息：

In [12]:
val sequence = Seq("a", 1, 0.0)
sequence.foreach { x =>
  x match {
    case s: String => println(s"$x is a String")
    case s: Int    => println(s"$x is an Int")
    case s: Double => println(s"$x is a Double")
    case _ => println(s"$x is an unknown type!")
  }
}

a is a String
1 is an Int
0.0 is a Double


sequence: Seq[Any] = List("a", 1, 0.0)

<span style="color:blue"> **示例：多种类型匹配** </span><br>
如果要匹配一个值是否具有多种类型中的某一个，请使用以下语法。 *请注意，匹配时**必须**使用`_`。*

In [13]:
val sequence = Seq("a", 1, 0.0)
sequence.foreach { x =>
  x match {
    case _: Int | _: Double => println(s"$x is a number!") // 是否整型或双精度
    case _ => println(s"$x is an unknown type!")
  }
}

a is an unknown type!
1 is a number!
0.0 is a number!


sequence: Seq[Any] = List("a", 1, 0.0)


<span style="color:blue"> **示例：类型匹配和类型擦除（Erasure）** </span><br>
类型匹配有一些限制。因为Scala在JVM上运行，并且JVM不维护多态类型，所以在运行时无法匹配它们（因为它们都被擦除）。请注意，以下示例始终与第一个case语句匹配，因为`[String]`，`[Int]`和`[Double]`的多态类型都被擦除掉了，所以case语句**实际上**仅仅匹配了一个`Seq`。

In [14]:
val sequence = Seq(Seq("a"), Seq(1), Seq(0.0))
sequence.foreach { x =>
  x match {
    case s: Seq[String] => println(s"$x is a String")
    case s: Seq[Int]    => println(s"$x is an Int")
    case s: Seq[Double] => println(s"$x is a Double")
  }
}

List(a) is a String
List(1) is a String
List(0.0) is a String


sequence: Seq[Seq[Any]] = List(List("a"), List(1), List(0.0))


请注意，如果出现类似上述示例中的代码，Scala编译器通常会发出警告。

<span style="color:blue">**示例：可选重置的匹配** </span><br>
下面的代码块显示了另一种方法实现的`DelayBy1`模块，这里使用匹配结构，而不是`if/else`。

In [15]:
class DelayBy1(resetValue: Option[UInt] = None) extends Module {
  val io = IO(new Bundle {
    val in  = Input( UInt(16.W))
    val out = Output(UInt(16.W))
  })
  val reg = resetValue match {
    case Some(r) => RegInit(r)
    case None    => Reg(UInt())
  }
  reg := io.in
  io.out := reg
}

println(getVerilog(new DelayBy1))
println(getVerilog(new DelayBy1(Some(3.U))))

[info] [0.000] Elaborating design...
[info] [0.008] Done elaborating.
Total FIRRTL Compile Time: 30.2 ms
module cmd14HelperDelayBy1(
  input         clock,
  input         reset,
  input  [15:0] io_in,
  output [15:0] io_out
);
  reg [15:0] reg_; // @[cmd14.sc 8:24]
  reg [31:0] _RAND_0;
  assign io_out = reg_; // @[cmd14.sc 11:10]
`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif
`ifndef RANDOM
`define RANDOM $random
`endif
`ifdef RANDOMIZE_MEM_INIT
  integer initvar;
`endif
initial begin
  `ifdef RANDOMIZE
    `ifdef INIT_RANDOM
      `INIT_RANDOM
    `endif
    `ifndef VERILATOR
      `ifdef RANDOMIZE_DELAY
        #`RANDOMIZE_DELAY begin end
      `else
        #0.002 begin end
      `endif
    `endif
  `ifdef RANDOMIZE_REG_INIT
  _RAND_0 = {1{`RANDOM}};
  reg_ = _RAND_0[15:0];
  `endif // RANDOMIZE_REG_INIT
  `endif /

defined class DelayBy1



---
# 带有可选字段的输入输出

有时我们希望包含或不包含某些IO。
也许有一些内部状态能够用来在调试中查看，但是你希望在生成整个系统的时候隐藏它。
也许你的生成器有一些输入，不需要在每种情况下都连接，因为它已经具有了一个合理的默认值。

<span style="color:blue"> **示例：使用Option的可选IO** </span><br>
Bundle中的可选字段是实现此功能的一种方法。
在下面的示例中，我们展示了一位可选择接收进位的一位加法器。
如果包含进位，`io.carryIn`将具有类型`Some [UInt]`并包含在IO bundle中。
如果不包含进位，则`io.carryIn`将具有类型`None`并将从IO包中排除。

In [16]:
class HalfFullAdder(val hasCarry: Boolean) extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(1.W))
    val b = Input(UInt(1.W))
    val carryIn = if (hasCarry) Some(Input(UInt(1.W))) else None
    val s = Output(UInt(1.W))
    val carryOut = Output(UInt(1.W))
  })
  val sum = io.a +& io.b +& io.carryIn.getOrElse(0.U)
  io.s := sum(0)
  io.carryOut := sum(1)
}

class HalfAdderTester(c: HalfFullAdder) extends PeekPokeTester(c) {
  require(!c.hasCarry, "DUT must be half adder")
  // 0 + 0 = 0
  poke(c.io.a, 0)
  poke(c.io.b, 0)
  expect(c.io.s, 0)
  expect(c.io.carryOut, 0)
  // 0 + 1 = 1
  poke(c.io.b, 1)
  expect(c.io.s, 1)
  expect(c.io.carryOut, 0)
  // 1 + 1 = 2
  poke(c.io.a, 1)
  expect(c.io.s, 0)
  expect(c.io.carryOut, 1)
  // 1 + 0 = 1
  poke(c.io.b, 0)
  expect(c.io.s, 1)
  expect(c.io.carryOut, 0)
}

class FullAdderTester(c: HalfFullAdder) extends PeekPokeTester(c) {
  require(c.hasCarry, "DUT must be half adder")
  poke(c.io.carryIn.get, 0)
  // 0 + 0 + 0 = 0
  poke(c.io.a, 0)
  poke(c.io.b, 0)
  expect(c.io.s, 0)
  expect(c.io.carryOut, 0)
  // 0 + 0 + 1 = 1
  poke(c.io.b, 1)
  expect(c.io.s, 1)
  expect(c.io.carryOut, 0)
  // 0 + 1 + 1 = 2
  poke(c.io.a, 1)
  expect(c.io.s, 0)
  expect(c.io.carryOut, 1)
  // 0 + 1 + 0 = 1
  poke(c.io.b, 0)
  expect(c.io.s, 1)
  expect(c.io.carryOut, 0)

  poke(c.io.carryIn.get, 1)
  // 1 + 0 + 0 = 1
  poke(c.io.a, 0)
  poke(c.io.b, 0)
  expect(c.io.s, 1)
  expect(c.io.carryOut, 0)
  // 1 + 0 + 1 = 2
  poke(c.io.b, 1)
  expect(c.io.s, 0)
  expect(c.io.carryOut, 1)
  // 1 + 1 + 1 = 3
  poke(c.io.a, 1)
  expect(c.io.s, 1)
  expect(c.io.carryOut, 1)
  // 1 + 1 + 0 = 2
  poke(c.io.b, 0)
  expect(c.io.s, 0)
  expect(c.io.carryOut, 1)
}

val worksHalf = iotesters.Driver(() => new HalfFullAdder(false)) { c => new HalfAdderTester(c) }
val worksFull = iotesters.Driver(() => new HalfFullAdder(true)) { c => new FullAdderTester(c) }
assert(worksHalf && worksFull) // Scala 代码：如果 works == false，这里会抛出异常
println("SUCCESS!!") // Scala 代码：到这里测试就通过了

[info] [0.000] Elaborating design...
[info] [0.049] Done elaborating.
Total FIRRTL Compile Time: 28.9 ms
Total FIRRTL Compile Time: 17.2 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1572042832758
test cmd15HelperHalfFullAdder Success: 8 tests passed in 5 cycles taking 0.007381 seconds
[info] [0.005] RAN 0 CYCLES PASSED
[info] [0.000] Elaborating design...
[info] [0.004] Done elaborating.
Total FIRRTL Compile Time: 22.1 ms
Total FIRRTL Compile Time: 14.9 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1572042832924
test cmd15HelperHalfFullAdder Success: 16 tests passed in 5 cycles taking 0.013992 seconds
[info] [0.013] RAN 0 CYCLES PASSED
SUCCESS!!


defined class HalfFullAdder
defined class HalfAdderTester
defined class FullAdderTester
worksHalf: Boolean = true
worksFull: Boolean = true


<span style="color:blue"> **示例：带有零宽度线的可选IO ** </span><br>
另一种方法是使用零宽度线来实现与Option相似功能的。
Chisel类型允许宽度为零。
在生成Verilog时宽度为零的IO会被删除，任何使用零宽度线的值的地方都会得到一个常数零。
如果零是合理的默认值，那么零宽度线可以很直观，因为它们不需要匹配一个option或调用`getOrElse`。

In [17]:
class HalfFullAdder(val hasCarry: Boolean) extends Module {
  val io = IO(new Bundle {
    val a = Input(UInt(1.W))
    val b = Input(UInt(1.W))
    val carryIn = Input(if (hasCarry) UInt(1.W) else UInt(0.W))
    val s = Output(UInt(1.W))
    val carryOut = Output(UInt(1.W))
  })
  val sum = io.a +& io.b +& io.carryIn
  io.s := sum(0)
  io.carryOut := sum(1)
}
println("Half Adder:")
println(getVerilog(new HalfFullAdder(false)))
println("\n\nFull Adder:")
println(getVerilog(new HalfFullAdder(true)))

Half Adder:
[info] [0.000] Elaborating design...
[info] [0.007] Done elaborating.
Total FIRRTL Compile Time: 35.9 ms
module cmd16HelperHalfFullAdder(
  input   clock,
  input   reset,
  input   io_a,
  input   io_b,
  output  io_s,
  output  io_carryOut
);
  wire [1:0] _T; // @[cmd16.sc 9:18]
  wire [2:0] sum; // @[cmd16.sc 9:26]
  assign _T = io_a + io_b; // @[cmd16.sc 9:18]
  assign sum = {{1'd0}, _T}; // @[cmd16.sc 9:26]
  assign io_s = sum[0]; // @[cmd16.sc 10:8]
  assign io_carryOut = sum[1]; // @[cmd16.sc 11:15]
endmodule



Full Adder:
[info] [0.000] Elaborating design...
[info] [0.004] Done elaborating.
Total FIRRTL Compile Time: 30.7 ms
module cmd16HelperHalfFullAdder(
  input   clock,
  input   reset,
  input   io_a,
  input   io_b,
  input   io_carryIn,
  output  io_s,
  output  io_carryOut
);
  wire [1:0] _T; // @[cmd16.sc 9:18]
  wire [1:0] _GEN_0; // @[cmd16.sc 9:26]
  wire [2:0] sum; // @[cmd16.sc 9:26]
  assign _T = io_a + io_b; // @[cmd16.sc 9:18]
  assign _GEN_0 = {{1

defined class HalfFullAdder


---
# Implicits
通常，您需要编程时需要大量的不重要但是又没它不行的代码。为了处理这种情况，Scala引入了**implicits**的概念，它允许编译器为你做一些语法糖（syntactic sugar，对语言的功能没有影响，但是更方便程序员使用的语法）。因为很多事情都发生在幕后，所以implicits可能看起来非常神奇。本节分解了一些基本示例，以解释它们的含义以及它们的常用用法。

## 隐含参数
有时，您的代码需要从很深的一系列函数调用中访问某个顶层变量。您可以使用隐式参数为您执行此操作，而不是在每个函数调用时手动传递此变量。

<span style="color:blue"> **示例：隐式传递的猫** </span><br>
在以下示例中，我们可以隐式或显式地传递cat的数量。

In [18]:
object CatDog {
  implicit val numberOfCats: Int = 3
  //implicit val numberOfDogs: Int = 5

  def tooManyCats(nDogs: Int)(implicit nCats: Int): Boolean = nCats > nDogs
    
  val imp = tooManyCats(2)    // 参数被隐式传递!
  val exp = tooManyCats(2)(1) // 参数被显式传递!
}
CatDog.imp
CatDog.exp

defined object CatDog
res17_1: Boolean = true
res17_2: Boolean = false


上面发生了什么事？首先，我们定义一个隐含值**numberOfCats**。在一个给定作用域内，**一个给定类型只能有一个的隐含值**。然后，我们定义了一个带有两个参数的函数;第一个是显式参数，第二个是隐式参数。当我们调用**tooManyCats**时，我们要么省略第二个隐式参数（让编译器为我们找到它），要么为它显式提供一个值（可以与隐含值不同）。

以下是隐含参数会*失败*的情况：
 - 在作用域中定义了给定类型的两个或多个隐含值
 - 编译器找不到函数调用所需的隐含值

<span style="color:blue"> **示例：隐式日志** </span><br>
下面代码块展示了如何使用隐含参数在Chisel生成器中实现日志（log）功能。

***注意：Scala中有更好的写日志的方法！***

In [19]:
sealed trait Verbosity
implicit case object Silent extends Verbosity
case object Verbose extends Verbosity

class ParameterizedWidthAdder(in0Width: Int, in1Width: Int, sumWidth: Int)(implicit verbosity: Verbosity)
extends Module {
  def log(msg: => String): Unit = verbosity match {
    case Silent =>
    case Verbose => println(msg)
  }
  require(in0Width >= 0)
  log(s"in0Width of $in0Width OK")
  require(in1Width >= 0)
  log(s"in1Width of $in1Width OK")
  require(sumWidth >= 0)
  log(s"sumWidth of $sumWidth OK")
  val io = IO(new Bundle {
    val in0 = Input(UInt(in0Width.W))
    val in1 = Input(UInt(in1Width.W))
    val sum = Output(UInt(sumWidth.W))
  })
  log("Made IO")
  io.sum := io.in0 + io.in1
  log("Assigned output")
}

println(getVerilog(new ParameterizedWidthAdder(1, 4, 5)))
println(getVerilog(new ParameterizedWidthAdder(1, 4, 5)(Verbose)))

[info] [0.000] Elaborating design...
[info] [0.009] Done elaborating.
Total FIRRTL Compile Time: 29.5 ms
module cmd18HelperParameterizedWidthAdder(
  input        clock,
  input        reset,
  input        io_in0,
  input  [3:0] io_in1,
  output [4:0] io_sum
);
  wire [3:0] _GEN_0; // @[cmd18.sc 23:20]
  wire [3:0] _T_1; // @[cmd18.sc 23:20]
  assign _GEN_0 = {{3'd0}, io_in0}; // @[cmd18.sc 23:20]
  assign _T_1 = _GEN_0 + io_in1; // @[cmd18.sc 23:20]
  assign io_sum = {{1'd0}, _T_1}; // @[cmd18.sc 23:10]
endmodule

[info] [0.000] Elaborating design...
in0Width of 1 OK
in1Width of 4 OK
sumWidth of 5 OK
Made IO
Assigned output
[info] [0.007] Done elaborating.
Total FIRRTL Compile Time: 34.9 ms
module cmd18HelperParameterizedWidthAdder(
  input        clock,
  input        reset,
  input        io_in0,
  input  [3:0] io_in1,
  output [4:0] io_sum
);
  wire [3:0] _GEN_0; // @[cmd18.sc 23:20]
  wire [3:0] _T_1; // @[cmd18.sc 23:20]
  assign _GEN_0 = {{3'd0}, io_in0}; // @[cmd18.sc 23:20]
 

defined trait Verbosity
defined object Silent
defined object Verbose
defined class ParameterizedWidthAdder


## 隐式转换
与隐含参数一样，隐式函数（也称为**隐式转换**）用于减少那些不重要但是又没它不行的代码。更具体地说，它们用于自动将一个Scala对象转换为另一个。

<span style="color:blue">**示例：隐式转换**</span><br>
在下面的例子中，我们有两个类，`Animal`和`Human`。 `Animal`有一个`species`字段，但`Human`没有。但是，通过实现隐式转换，我们可以在`Human`上调用`species`。

In [20]:
class Animal(val name: String, val species: String)
class Human(val name: String)
implicit def human2animal(h: Human): Animal = new Animal(h.name, "Homo sapiens")
val me = new Human("Adam")
println(me.species)

Homo sapiens


defined class Animal
defined class Human
defined function human2animal
me: Human = ammonite.$sess.cmd19$Helper$Human@6d9f8e89


通常，implicits可能会使您的代码混乱，因此我们建议您将它们作为最后的手段使用。首先尝试继承，特征（traits）或方法重载。

---
# Generator示例
以下示例显示了具有1位输入Mealy机的生成器。
它有一个测试来自于[Wikipedia](https://en.wikipedia.org/wiki/Mealy_machine#/media/File:Mealy.png)里的示例。
仔细阅读代码并看看发生了什么。

<span style="color:blue">**示例：Mealy Machine ** </span><br>
尝试对Mealy机生成器进行别的参数化，并在下面的代码块中编写自己的测试。

In [21]:
// Mealy 机的参数
case class BinaryMealyParams(
  // 状态的数目
  nStates: Int,
  // 初始状态
  s0: Int,
  // 状态转移函数
  stateTransition: (Int, Boolean) => Int,
  // 输出函数
  output: (Int, Boolean) => Int
) {
  require(nStates >= 0)
  require(s0 < nStates && s0 >= 0)
}

class BinaryMealy(val mp: BinaryMealyParams) extends Module {
  val io = IO(new Bundle {
    val in = Input(Bool())
    val out = Output(UInt())
  })

  val state = RegInit(UInt(), mp.s0.U)

  // 没有任何状态时输出 0
  io.out := 0.U
  for (i <- 0 until mp.nStates) {
    when (state === i.U) {
      when (io.in) {
        state  := mp.stateTransition(i, true).U
        io.out := mp.output(i, true).U
      }.otherwise {
        state  := mp.stateTransition(i, false).U
        io.out := mp.output(i, false).U
      }
    }
  }
}

// 来自 https://en.wikipedia.org/wiki/Mealy_machine 的示例
val nStates = 3
val s0 = 2
def stateTransition(state: Int, in: Boolean): Int = {
  if (in) {
    1
  } else {
    0
  }
}
def output(state: Int, in: Boolean): Int = {
  if (state == 2) {
    return 0
  }
  if ((state == 1 && !in) || (state == 0 && in)) {
    return 1
  } else {
    return 0
  }
}

val testParams = BinaryMealyParams(nStates, s0, stateTransition, output)

class BinaryMealyTester(c: BinaryMealy) extends PeekPokeTester(c) {
  poke(c.io.in, false)
  expect(c.io.out, 0)
  step(1)
  poke(c.io.in, false)
  expect(c.io.out, 0)
  step(1)
  poke(c.io.in, false)
  expect(c.io.out, 0)
  step(1)
  poke(c.io.in, true)
  expect(c.io.out, 1)
  step(1)
  poke(c.io.in, true)
  expect(c.io.out, 0)
  step(1)
  poke(c.io.in, false)
  expect(c.io.out, 1)
  step(1)
  poke(c.io.in, true)
  expect(c.io.out, 1)
  step(1)
  poke(c.io.in, false)
  expect(c.io.out, 1)
  step(1)
  poke(c.io.in, true)
  expect(c.io.out, 1)
}
val works = iotesters.Driver(() => new BinaryMealy(testParams)) { c => new BinaryMealyTester(c) }
assert(works) // Scala 代码：如果 works == false，这里会抛出异常
println("SUCCESS!!") // Scala 代码：到这里测试就通过了

[info] [0.000] Elaborating design...
[info] [0.054] Done elaborating.
Total FIRRTL Compile Time: 29.2 ms
Total FIRRTL Compile Time: 25.8 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1572042835624
test cmd20HelperBinaryMealy Success: 9 tests passed in 13 cycles taking 0.011317 seconds
[info] [0.009] RAN 8 CYCLES PASSED
SUCCESS!!


defined class BinaryMealyParams
defined class BinaryMealy
nStates: Int = 3
s0: Int = 2
defined function stateTransition
defined function output
testParams: BinaryMealyParams = BinaryMealyParams(
  3,
  2,
  ammonite.$sess.cmd20$Helper$$Lambda$5434/0x00000008415de040@72e456fa,
  ammonite.$sess.cmd20$Helper$$Lambda$5435/0x00000008415df040@b0df437
)
defined class BinaryMealyTester
works: Boolean = true

---
# 本节结束!

[返回顶部](#top)